In [3]:
def genEphemeralParameter(n, g):
    s = randint(1, n)
    while True:
        parameter = s * g
        s = randint(1, n)
        if 0 < int(parameter[0]) * 256 + int(parameter[1]) < n:
            return parameter

In [3]:
# def getParameterLastSession():

In [4]:
def calculate_sha256(data):
    if isinstance(data,str):
        data=data.encode()
    sha256_hash=hashlib.sha256(data).hexdigest()
    return sha256_hash

In [5]:
def _convert_to_bytes(digest_input: Union[bytes, str], arg_name: str) -> bytes:
    """
    Helper function to convert input (hex string or bytes) to validated bytes.

    Args:
        digest_input: The input digest (bytes or hex string).
        arg_name: The name of the argument being processed (for error messages).

    Returns:
        The digest as a bytes object.

    Raises:
        TypeError: If input is not bytes or string.
        ValueError: If string input is not valid hex, or if the resulting
                    bytes are not exactly 32 bytes long.
    """
    _SHA256_DIGEST_LENGTH = 32
    if isinstance(digest_input, str):
        if len(digest_input) != _SHA256_DIGEST_LENGTH * 2:
             raise ValueError(
                f"Argument '{arg_name}' (string) must be "
                f"{_SHA256_DIGEST_LENGTH * 2} characters long for a SHA-256 hex digest, "
                f"but got {len(digest_input)} characters."
            )
        try:
            digest_bytes = bytes.fromhex(digest_input)
        except ValueError:
            raise ValueError(
                f"Argument '{arg_name}' is a string but not valid hexadecimal."
            ) from None # Suppress original error chain for clarity
    elif isinstance(digest_input, bytes):
        if len(digest_input) != _SHA256_DIGEST_LENGTH:
            raise ValueError(
                f"Argument '{arg_name}' (bytes) must be {_SHA256_DIGEST_LENGTH} bytes long "
                f"(SHA-256 digest length), but got {len(digest_input)} bytes."
            )
        digest_bytes = digest_input
    else:
        raise TypeError(
            f"Argument '{arg_name}' must be bytes or a hex string, "
            f"not {type(digest_input).__name__}"
        )

    # This check is technically redundant now due to checks above, but is a good safeguard
    # assert len(digest_bytes) == _SHA256_DIGEST_LENGTH, "Internal length validation failed"
    return digest_bytes

In [6]:
def xor_sha256_digests(digest1: Union[bytes, str], digest2: Union[bytes, str]):
    bytes1 = _convert_to_bytes(digest1, 'digest1')
    bytes2 = _convert_to_bytes(digest2, 'digest2')

    xor_result = bytes(b1 ^^ b2 for b1, b2 in zip(bytes1, bytes2))

    return xor_result.hex()

In [7]:
def create_ASCII_map(parameter, g):
    ASCIIMap = []
    for i in range(256):
        ASCIIMap.append((i, (parameter[0] * i + parameter[1]) * g))
    return ASCIIMap

In [8]:
def encode(message, table):
    encodeMess = []
    for letter in message:
        encodeMess.append(table[ord(letter)][1])
    return encodeMess

In [9]:
def decode(Pm, table):
    message = []
    for i in Pm:
        for j in table:
            if i == j[1]:
                message.append(j[0])
    return ''.join(map(chr,message))

In [10]:
def createRandomMessage(bit, quantity):
    list = []
    for i in range(quantity):
        characters = string.ascii_letters + string.digits
        random_string = ''.join(random.choices(characters, k= bit / 8))
        list.append(random_string)
    return list

4.1. α and β generation phase <br>
 In this step, Alice use the point generator G and the order of the selected curve.<br>
 (i) Alice chooses randomly an integer s where s [ [0, n - 1].<br>
 (ii) Alice performs s∗G = (x, y).<br>
 (iii) Alice verifies x and y with the following condition: ((x* Max ASCII Code) + y) ∈ [0, n - 1] where Max ASCII Code=256.<br>
 (iv) If x and y verify the condition, then a = x and b = y , else we go to step 1 and repeat the process until we get the best α and β.

In [11]:
p = 100000000000000000039
a = 0
b = 7

In [13]:
E = EllipticCurve(GF(p),[a, b])
G = E.gens()
g = E.point(G[0])
n = g.order(); g

(69496283786894253888 : 58916120834459954240 : 1)

In [14]:
n

50000000008969935366

In [15]:
kA = randint(1, n)
QA = kA * g

kB = randint(1, n)
QB = kB * g

In [16]:
kA

17234869679141697959

In [17]:
kB

32426097915697275469

In [18]:
QA

(24148284530724724990 : 89237694802055227180 : 1)

In [19]:
QB

(67119382834134792199 : 71328156358363944895 : 1)

In [20]:
ephemeral = genEphemeralParameter(n, g); ephemeral

(125929319290300242 : 7491963579492646982 : 1)

In [21]:
ephemeralLastSession = genEphemeralParameter(n, g); ephemeralLastSession

(44522709707356346 : 23226884570579358018 : 1)

 4.2. α and β transmission and the mutual authentication phase 
 Alice Side:
 - First, Alice selects a random integer r, then performs C1 = r ∗ G and C2 = r ∗ QB +(α,β) to get C = (C1, C2).
 - Second, Alice performs h1=hash (C ∥ (α, β)lastsession ), (α, β)lastsession are the encoding parameters of last session.
 - Finally, Alice sends Bob C and h1

In [22]:
r = randint(1, n)
C1 = r * g
C2 = r * QB + ephemeral
C = (C1, C2)
h1 = calculate_sha256(f"{C}|{ephemeralLastSession}")

In [23]:
C1

(5405829232180582341 : 72620887608343965895 : 1)

In [24]:
h1

'1bad007a3c02584516caa7d93a89bd0393147876500925e45810fa53f17226a7'

In [25]:
C2

(83191304044977708726 : 9188903516077432223 : 1)

Bob Side:
 When Bob gets the message:
 - First, Bob decrypts C to get (α, β) by computing M = KB∗C1 = r∗KB∗G the C2 −M=r∗QB+(a,b)−r∗KB∗G =(a, b).
 - Second Bob calculates h2= hash (C ∥ (α, β)lastsession ) and compares h1 and h2.
 - If h1 match h2, Bob authenticates Alice and he concludes the integrity of (α, β) and then generates the ASCII map table using α and β.
 - Third, Bob calculates h3=hash ((α, β)) then performs h3 ⊕ h1 = CH1.
 - Finally, Bob sends CH1 to Alice.

In [26]:
M = kB * C[0]
parameterB = C[1] - M
h2 = calculate_sha256(f"{C}|{ephemeralLastSession}")
if h1 == h2:
    tableB = create_ASCII_map(parameterB, g)
    h3 = calculate_sha256(f"{parameterB}")
    CH1 = xor_sha256_digests(h1, h3)

In [30]:
M

(20167080408120019994 : 73521288540533939090 : 1)

In [27]:
h2

'1bad007a3c02584516caa7d93a89bd0393147876500925e45810fa53f17226a7'

In [28]:
h3

'bc3a9368484267de0881392b4535cbb7f31246ba28010caa82f75696f1ececd1'

In [29]:
CH1

'a797931274403f9b1e4b9ef27fbc76b460063ecc7808294edae7acc5009eca76'

Alice Side: When Alice receives CH1:
 - First, she executes h4= hash ((α, β)).
 - Then, she performs CH2= h1 ⊕ CH1. if CH2 matches h4, Alice authenticates Bob and she is sure that Bob get the correct (α, β).
 - After that, she uses α and β to generate the ASCII-map table

In [31]:
h4 = calculate_sha256(f"{ephemeral}")
CH2 = xor_sha256_digests(h1, CH1)
if CH2 == h4:
    print("authenticate")
    tableA = create_ASCII_map(ephemeral, g)

authenticate


In [32]:
h4

'bc3a9368484267de0881392b4535cbb7f31246ba28010caa82f75696f1ececd1'

In [33]:
CH2

'bc3a9368484267de0881392b4535cbb7f31246ba28010caa82f75696f1ececd1'

1. Tấn công Man-in-the-Middle (MitM) cố gắng thay đổi C <br>
Kịch bản tấn công: Giả định kẻ tấn công (Eve) có khả năng chặn và sửa đổi thông điệp trao đổi giữa Alice và Bob. Khi Alice gửi cặp (C, h1) cho Bob, Eve can thiệp, thay đổi C = (C1, C2) thành C′ = (C′1, C′2) nhưng giữ nguyên h1 (hoặc không thể tính
toán h′1 tương ứng với C′ và (α, β)lastsession do không sở hữu (α, β)lastsession). Eve sau đó chuyển tiếp (C′, h1) đến Bob.

In [86]:
C1fake = E.point((30824315891190531571 , 16191521185314524171))
C2fake = E.point((62242932543577517545 , 93029969937313491867 ))
Cfake = (C1fake, C2fake )
h2_Mith = calculate_sha256(f"{Cfake}|{ephemeralLastSession}")

In [87]:
h2_Mith

'cdc27ff9680cd0abeedf6cbcd037325482bf1b1cd210535047c148dcb60c18c5'

2. Tấn công phát lại (Replay Attack) đơn giản <br>
Kịch bản tấn công: Kẻ tấn công Eve thu thập một cặp (C, h1) hợp lệ từ một phiên trao đổi trước đó giữa Alice và Bob. Ở một phiên làm việc sau, Eve cố gắng gửi lại cặp (C, h1) này cho Bob.

In [34]:
h1

'1bad007a3c02584516caa7d93a89bd0393147876500925e45810fa53f17226a7'

In [35]:
h2_Replay = calculate_sha256(f"{C}|{parameterB}"); h2_Replay

'3af7e25e7557dbba91cfbfcc81cdd24f9019790066d610cd90224b4db12b7c5a'

3. Mô phỏng kẻ tấn công không có khóa bí mật của Bob cố gắng giả mạo Bob <br>
Kịch bản tấn công: Eve chặn cặp (C, h1) do Alice gửi. Eve không sở hữu khóa bí mật kB của Bob, do đó không thể giải mã C để thu được cặp (α, β) chính xác mà Alice đã mã hóa. Để giả mạo Bob, Eve tạo ra một thông điệp CH′1 gửi lại cho Alice.
Giả sử Eve tạo ra một cặp (α′, β′) ngẫu nhiên hoặc dựa trên một phỏng đoán nào đó và tính h′3 = hash((α′, β′)). Sau đó, Eve tính CH′1 = h′3 ⊕ h1 (trong đó h1 là giá trị Eve chặn được) và gửi CH′1 cho Alice.

In [36]:
fakeparameter = randint(1, n) * g; fakeparameter

(10933699806426560872 : 75750767755776949727 : 1)

In [37]:
h3fake = calculate_sha256(f"{fakeparameter}"); h3fake

'42234c900c2b650b9504621e4fa3d85206ba49c3d93c2d46751f573589a5ad4d'

In [38]:
CH1fake = xor_sha256_digests(h1, h3fake); CH1fake

'598e4cea30293d4e83cec5c7752a655195ae31b5893508a22d0fad6678d78bea'

In [39]:
CH2S = xor_sha256_digests(h1, CH1fake); CH2S

'42234c900c2b650b9504621e4fa3d85206ba49c3d93c2d46751f573589a5ad4d'

In [40]:
h4

'bc3a9368484267de0881392b4535cbb7f31246ba28010caa82f75696f1ececd1'

In [ ]:
def encrytion(message, n, g, QB, table):
    randomNum = randint(1, n)
    Pm = encode(message, table)
    # encrytion
    E = []
    E1 = randomNum * g
    for pm in Pm:
        E2 = randomNum * QB + pm
        E.append((E1, E2))
    return E

In [ ]:
def decrytion(E, table, kB):
    Pm = []
    for e in E:
        M = kB * e[0]
        Pm.append(e[1] - M)
    return decode(Pm, table)

In [ ]:
def test(bit, quantity):
    encrytionTime = 0
    decrytionTime = 0

    messages = createRandomMessage(bit, quantity)
    ephemeral = genEphemeralParameter(n, g)
    ephemeralLastSession = ephemeral 
            
    r = randint(1, n)
    C1 = r * g
    C2 = r * QB + ephemeral
    C = (C1, C2)
    h1 = calculate_sha256(f"{C}|{ephemeralLastSession}")
            
    M = kB * C[0]
    parameterB = C[1] - M
    h2 = calculate_sha256(f"{C}|{ephemeralLastSession}")
    if h1 == h2:
        tableB = createASCIIMap(parameterB, g)
        h3 = calculate_sha256(f"{parameterB}")
        CH1 = xor_sha256_digests(h1, h3)
            
            
    h4 = calculate_sha256(f"{ephemeral}")
    CH2 = xor_sha256_digests(h1, CH1)
    if CH2 == h4:
        tableA = createASCIIMap(ephemeral, g)
   
    
    timeTaken = 0
    for i in messages:
        begin = time.time_ns()
        ciphertext = encrytion(i, n, g, QB, tableA)
        end1 = time.time_ns()
        begin2 = time.time_ns()
        d = decrytion(ciphertext, tableB, kB)
        end2 = time.time_ns()
        encrytionTime += end1 - begin
        decrytionTime += end2 - begin2
    return (encrytionTime / quantity, decrytionTime / quantity)

In [ ]:
def testAuth():
    timeTaken = 0
    for i in range(100):
        begin = time.time()
        # secp192r1
        # p = int("FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFFFFFFFFFFFF", base = 16)
        # a = int("FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFFFFFFFFFFFC", base = 16)
        # b = int("64210519E59C80E70FA7E9AB72243049FEB8DEECC146B9B1", base = 16)
    
        # secp224r1 OK
        # p = int("FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF000000000000000000000001", base = 16)
        # a = int("FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFFFFFFFFFFFFFFFFFFFE", base = 16)
        # b = int("B4050A850C04B3ABF54132565044B0B7D7BFD8BA270B39432355FFB4", base = 16)
        
        # # secp256r1 OK
        # p = int("FFFFFFFF00000001000000000000000000000000FFFFFFFFFFFFFFFFFFFFFFFF", base = 16)
        # a = int("FFFFFFFF00000001000000000000000000000000FFFFFFFFFFFFFFFFFFFFFFFC", base = 16)
        # b = int("5AC635D8AA3A93E7B3EBBD55769886BC651D06B0CC53B0F63BCE3C3E27D2604B", base = 16)
        
        # # Secp384r1 OK
        p = int("FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFFFF0000000000000000FFFFFFFF", base = 16)
        a = int("FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFFFF0000000000000000FFFFFFFC", base = 16)
        b = int("B3312FA7E23EE7E4988E056BE3F82D19181D9C6EFE8141120314088F5013875AC656398D8A2ED19D2A85C8EDD3EC2AEF", base = 16)
        E = EllipticCurve(GF(p),[a, b])
        G = E.gens()
        g = E.point(G[0])
        n = g.order()
        kA = randint(1, n)
        QA = kA * g
        
        kB = randint(1, n)
        QB = kB * g
        ephemeral = genEphemeralParameter(n, g)
        ephemeralLastSession = ephemeral 
            
        r = randint(1, n)
        C1 = r * g
        C2 = r * QB + ephemeral
        C = (C1, C2)
        h1 = calculate_sha256(f"{C}|{ephemeralLastSession}")
            
        M = kB * C[0]
        parameterB = C[1] - M
        h2 = calculate_sha256(f"{C}|{ephemeralLastSession}")
        if h1 == h2:
            tableB = createASCIIMap(parameterB, g)
            h3 = calculate_sha256(f"{parameterB}")
            CH1 = xor_sha256_digests(h1, h3)
            
            
        h4 = calculate_sha256(f"{ephemeral}")
        CH2 = xor_sha256_digests(h1, CH1)
        if CH2 == h4:
            tableA = createASCIIMap(ephemeral, g)
        end = time.time()
        timeTaken += end - begin

    return round(timeTaken / 100, ndigits=4)

In [ ]:
testAuth()

## RSA

In [ ]:
n = 7680
lower_bound = 2**(n - 1)
upper_bound = 2**n - 1 # Số lớn nhất có n bit

p = random_prime(upper_bound,False,lower_bound)

In [ ]:
def encodeRSA(message):
    encoded = []
    for i in message:
        encoded.append(ord(i))
    return encoded

In [ ]:
def decodeRSA(encode):
    return ''.join(map(chr,encode))

In [ ]:
def encrytionRSA(message, e, n):
    encodeText = encodeRSA(message)
    cipherText = []
    for i in encodeText:
        C = power_mod(i, e, n)
        cipherText.append(C)
    return cipherText

In [ ]:
def decrytionRSA(cipher, d, n):
    plain = []
    for i in cipher:
        D = power_mod(i, d, n)
        plain.append(D)
    return decodeRSA(plain)

In [ ]:
def test_rsa(bit, messageBit, quantity):
    

    message = createRandomMessage(messageBit, quantity)
    encrytionTime = 0
    decrytionTime = 0
    lower_bound = 2**(bit - 1)
    upper_bound = 2**bit - 1 # Số lớn nhất có n bit

    p = random_prime(upper_bound,False,lower_bound)
    q = random_prime(upper_bound,False,lower_bound)
    
    n = p * q
    phi_n = (p - 1) * (q - 1)
    
    
    for m in message:
        e = randint(1, phi_n)
        while gcd(e, phi_n) != 1:
            e = randint(1, phi_n)
        d = inverse_mod(e, phi_n)
        
        begin = time.time()
        E = encrytionRSA(m, e, n)
        end1 = time.time()
        
        D = decrytionRSA(E, d, n)
        end2 = time.time()
        encrytionTime += end1 - begin
        decrytionTime += end2 - end1
    return (encrytionTime / quantity, decrytionTime / quantity)

## Test

In [ ]:
testECEG = test(8, 100); testECEG

In [ ]:
timeECEG = []

In [ ]:
timeECEG.append(testECEG)

In [ ]:
testRSA = test_rsa(7980, 256, 30); testRSA

In [ ]:
timeRSA = []

In [ ]:
timeRSA.append(testRSA)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# --- Hàm để thêm nhãn dữ liệu ---
def autolabel(rects, vertical_offset=0.2):
    """Gắn nhãn văn bản phía trên mỗi thanh trong *rects*, hiển thị chiều cao của nó."""
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width() / 2.,  # Vị trí X (tâm thanh)
                height + vertical_offset,              # Vị trí Y (trên thanh + offset)
                f'{height:.1f}',                      # Văn bản nhãn (định dạng 1 chữ số thập phân)
                ha='center', va='bottom',              # Căn chỉnh ngang và dọc
                fontsize=9)                           # Kích thước phông chữ (tùy chọn)
# ---------------------------------

# 1. Dữ liệu (Ước tính từ hình ảnh của bạn - **Sử dụng dữ liệu thực tế của bạn**)
key_sizes = [112, 128, 192]  # Giá trị trục X
eceg_times = [0.2169, 0.2229, 0.4316]  # Chiều cao thanh màu xanh lam
rsa_times = [0.0254, 0.0696, 0.7228] # Chiều cao thanh màu đỏ

# 2. Thiết lập vị trí thanh trên trục X
x = np.arange(len(key_sizes))  # Vị trí nhãn (0, 1, 2, 3)
width = 0.2  # Chiều rộng của MỖI thanh
gap = 0.05    # Khoảng cách mong muốn giữa thanh xanh và đỏ trong một nhóm

# Tính toán vị trí tâm mới cho mỗi thanh để tạo khoảng cách
offset = width / 2 + gap / 2

# 3. Tạo biểu đồ
fig, ax = plt.subplots(figsize=(8, 5)) # Bạn có thể điều chỉnh kích thước nếu cần

# Vẽ các thanh tại các vị trí đã được điều chỉnh
rects1 = ax.bar(x - offset, eceg_times, width, label='Enhanced ECEG Total Time', color='tab:blue')
rects2 = ax.bar(x + offset, rsa_times, width, label='RSA Total Time', color='tab:red')

# 4. Thêm nhãn, tiêu đề và chú giải
ax.set_ylabel('Total Time (ms)') # Thêm nhãn trục Y nếu bạn muốn
# ax.set_xlabel('Key Size') # Thêm nhãn trục X nếu bạn muốn
ax.set_title('8-bits')
ax.set_xticks(x)
ax.set_xticklabels(key_sizes) # Đặt nhãn trục X thành kích thước khóa thực tế

# Đặt giới hạn và dấu kiểm trục Y để khớp với hình ảnh gốc
ax.set_ylim(0, 1)
ax.set_yticks(np.arange(0, 1, 0.1)) # Tạo dấu kiểm từ 0 đến 16 với khoảng cách 2

# Thêm đường lưới ngang mờ nhạt tương tự như bản gốc
ax.yaxis.grid(True, linestyle='-', linewidth=0.5, color='lightgrey')
ax.set_axisbelow(True) # Đảm bảo lưới nằm sau các thanh
# --- Gọi hàm autolabel cho cả hai bộ thanh ---
autolabel(rects1)
autolabel(rects2)
# -------------------------------------------
# Thêm chú giải
ax.legend()

# 5. Hiển thị biểu đồ
plt.tight_layout() # Điều chỉnh bố cục để tránh chồng chéo
fig1 = plt.gcf()
plt.show()
plt.draw()
fig1.savefig('8bit.png', dpi=100)

In [ ]:
timeRSA